# Basic AoI visualization

## Load a shapefile or geojson

In [ ]:
import fsspec

fs = fsspec.filesystem('s3', requester_pays=True)

In [ ]:
shape_paths = fs.ls('chs-pangeo-data-bucket/klsmith/misc')
shape_paths

In [ ]:
import zipfile

with fs.open('chs-pangeo-data-bucket/klsmith/misc/CA-senay-buffered.zip') as f:
    with zipfile.ZipFile(f) as zf:
        print(zf.namelist())

In [ ]:
import os

import boto3
import geopandas as gpd
import fiona

_S3SESSION = fiona.session.AWSSession(boto3.Session(), requester_pays=True)

def load_polygon(path: str) -> gpd.geodataframe.GeoDataFrame:
    """
    Load a polygon file into a GeoDataFrame
    """
    with fiona.Env(session=_S3SESSION):
        return gpd.read_file(path)

In [ ]:
load_polygon('zip+s3://chs-pangeo-data-bucket/klsmith/misc/CA-senay-buffered.zip')

In [ ]:
def s3zipped_shapefile(key: str) -> gpd.geodataframe.GeoDataFrame:
    """
    Load a zipped shapefile from s3, given the s3 key
    """
    return load_polygon('zip+s3://' + key)

In [ ]:
ca_buffered = s3zipped_shapefile(shape_paths[2])
ca_buffered

In [ ]:
ca_buffered.crs

## Simple plotting (ESRI data, etc ...)

In [ ]:
import geoviews as gv
from geoviews import tile_sources as gvts
from cartopy import crs

gv.extension('bokeh', 'matplotlib')

In [ ]:
gv.Layout([ts.relabel(name) for name, ts in gvts.tile_sources.items()]).opts('WMTS', xaxis=None, yaxis=None, width=225, height=225).cols(4)

In [ ]:
viz_base = gv.tile_sources.EsriImagery.opts(width=650, height=500)
viz_base

In [ ]:
viz_ca_buffered = gv.Polygons(ca_buffered, crs=crs.epsg(5070)).opts(line_color='red', color=None)
viz_ca_buffered

In [ ]:
viz_base * viz_ca_buffered

# STAC and STAC-API

 - What is STAC
 - What is STAC-API
 - Interacting with Collection 2 STAC-API

## What is STAC

<img src="https://stacspec.org/images/logo/STAC-03.png" alt="STAC" width="200"/>

From the STAC website (https://stacspec.org/):  

"The SpatioTemporal Asset Catalog (STAC) specification provides a common language to describe a range of geospatial information, so it can more easily be indexed and discovered. A 'spatiotemporal asset' is any file that represents information about the earth captured in a certain space and time.

The goal is for all providers of spatiotemporal assets (Imagery, SAR, Point Clouds, Data Cubes, Full Motion Video, etc) to expose their data as SpatioTemporal Asset Catalogs (STAC), so that new code doesn't need to be written whenever a new data set or API is released."

**STAC Specification**  

The STAC Specification consists of 4 semi-independent specifications. Each can be used alone, but they work best in concert with one another.

 - STAC Item is the core atomic unit, representing a single spatiotemporal asset as a GeoJSON feature plus datetime and links.
 - STAC Catalog is a simple, flexible JSON file of links that provides a structure to organize and browse STAC Items. A series of best practices helps make recommendations for creating real world STAC Catalogs.
 - STAC Collection is an extension of the STAC Catalog with additional information such as the extents, license, keywords, providers, etc that describe STAC Items that fall within the Collection.

So ...? What is STAC?

STAC at it's core is standardized metadata that can be linked together. That's mostly it.
 - Provides standardized spatial and temporal representations
 - Link together assets to define the entire data set
 - Maybe helps with data discovery? (debatable, who wants to crawl linked lists these days?)
 - Can really help with provenance tracking
 
Does not replace our beloved FGDC metadata ...

In [ ]:
import requests

_stac = 'https://landsatlook.usgs.gov/sat-api/stac'

In [ ]:
requests.get(_stac).json()

In [ ]:
requests.get('https://landsatlook.usgs.gov/sat-api/collections/landsat-c2l2-sr').json()

In [ ]:
# resp = requests.get('https://landsatlook.usgs.gov/sat-api/collections/landsat-c2l2alb-sr/items').json()
resp = requests.get('https://landsatlook.usgs.gov/sat-api/collections/landsat-c2l2-sr/items').json()
resp.keys()

In [ ]:
print(resp['type'], '\n',
      resp['meta'], '\n',
      resp['links'])

In [ ]:
resp['features'][0]

In [ ]:
from IPython.display import Image

with fs.open(resp['features'][0]['assets']['thumbnail']['href'].replace('https://landsatlook.usgs.gov/data', 's3://usgs-landsat')) as f:
    data = f.read()

Image(data=data)

## What is STAC-API

"STAC API provides a RESTful endpoint that enables search of STAC Items, specified in OpenAPI, following OGC's WFS 3."

So ...? What is STAC-API?

Take the standardized metadata, shove it into a database, and then enable standardized querying of the data through HTTP.

https://landsatlook.usgs.gov/sat-api/stac  
https://landsatlook.usgs.gov/stac-browser

Resources:  
https://github.com/radiantearth/stac-api-spec/blob/master/item-search/README.md  
https://medium.com/pangeo/intake-stac-nasa-4cd78d6246b7  
https://github.com/sat-utils/sat-search

In [ ]:
from functools import lru_cache
from typing import Union

_stac = 'https://landsatlook.usgs.gov/sat-api/stac'

def get_json(url: str, params: Union[list, dict, None] = None) -> Union[list, dict]:
    """
    Perform a GET request to a REST API endpoint, assume the response is a JSON
    """
    resp = requests.get(url, params=params)
    
    if not resp.ok:
        resp.raise_for_status()

    return resp.json()

def post_json(url: str, data: Union[list, dict]) -> Union[list, dict]:
    """
    Send a POST request to the resource with the payload as JSON
    """
    resp = requests.post(url, json=data)
    
    if not resp.ok:
        resp.raise_for_status()
        
    return resp.json()

@lru_cache()
def search_endpoint(endpoint: str = _stac) -> str:
    """
    Get the search endpoint for STAC
    """
    resp = get_json(endpoint)
    
    for item in resp['links']:
        if item['rel'] == 'search':
            return item['href']

def stac_search(params: dict, endpoint: str = _stac) -> Union[list, dict]:
    """
    Simple search against STAC-API
    """
    return get_json(search_endpoint(endpoint),
                    params=params)

def stac_query(data: dict, endpoint: str = _stac) -> Union[list, dict]:
    """
    Perform a more complex query against the STAC-API
    """
    return post_json(search_endpoint(endpoint),
                     data=data)

In [ ]:
items = stac_search({
    'bbox': f"[{','.join(str(_) for _ in ca_buffered.to_crs(4326)['geometry'][0].bounds)}]",
    'limit': 100,
    'time': '2018-07-01/2018-08-01',
    'collection': 'landsat-c2l2-sr'
})
items.keys()

In [ ]:
len(items['features'])

In [ ]:
items = stac_query({
     'bbox': f"[{','.join(str(_) for _ in ca_buffered.to_crs(4326)['geometry'][0].bounds)}]",
     'limit': 100,
     'time': '2018-07-01/2018-09-01',
     'query': {'eo:cloud_cover': {'lt': 10},
               'eo:instrument': {'eq': 'OLI_TIRS'}}
 })
len(items['features'])

In [ ]:
from collections import Counter

Counter([(f['properties']['landsat:wrs_path'], f['properties']['landsat:wrs_row'])
         for f in items['features']])

In [ ]:
 items = stac_query({
     'bbox': f"[{','.join(str(_) for _ in ca_buffered.to_crs(4326)['geometry'][0].bounds)}]",
     'limit': 100,
     'time': '2018-07-01/2018-09-01',
     'query': {'eo:cloud_cover': {'lt': 10},
               'eo:instrument': {'eq': 'OLI_TIRS'},
               'landsat:wrs_path': {'lt': 100},
               'landsat:wrs_row': {'lt': 100}}
 })
len(items['features'])

In [ ]:
 items = stac_query({
     'bbox': f"[{','.join(str(_) for _ in ca_buffered.to_crs(4326)['geometry'][0].bounds)}]",
     'limit': 100,
     'time': '2018-07-01/2018-09-01',
     'query': {'eo:cloud_cover': {'lt': 10},
               'eo:instrument': {'eq': 'OLI_TIRS'},
               'landsat:wrs_path': {'eq': 42},
               'landsat:wrs_row': {'eq': 35}}
 })
len(items['features'])

In [ ]:
from cytoolz import pipe
from typing import List, Callable

def filter_ascending(features: List[dict]) -> List[dict]:
    """
    Filter out ascending path/rows from a STAC query
    """
    return [f for f in features
            if int(f['properties']['landsat:wrs_path']) < 100
               and int(f['properties']['landsat:wrs_row']) < 100]

def filter_stacquery(data: dict, filters: Union[List[Callable], Callable], endpoint: str = _stac) -> List[dict]:
    """
    Conduct a STAC query, then apply some filters to the results
    """
    items = stac_query(data, endpoint)

    if isinstance(filters, Callable):
        return filters(items['features'])
    else:
        return pipe(items['features'], *filters)

In [ ]:
filtered = filter_stacquery({
    'bbox': f"[{','.join(str(_) for _ in ca_buffered.to_crs(4326)['geometry'][0].bounds)}]",
    'limit': 100,
    'time': '2018-07-01/2018-09-01',
    'query': {'eo:cloud_cover': {'lt': 10},
              'eo:instrument': {'eq': 'OLI_TIRS'}}},
    filters=filter_ascending)

len(filtered)

In [ ]:
Counter([(f['properties']['landsat:wrs_path'], f['properties']['landsat:wrs_row'])
         for f in filtered])

In [ ]:
filtered[0]

In [ ]:
with fs.open(filtered[0]['assets']['SR_B1.TIF']['href'].replace('https://landsatlook.usgs.gov/data', 's3://usgs-landsat')) as f:
    data = f.read()

Image(data=data)

In [ ]:
with fs.open(filtered[0]['assets']['thumbnail']['href'].replace('https://landsatlook.usgs.gov/data', 's3://usgs-landsat')) as f:
    data = f.read()

Image(data=data)

# Single scene extraction

 - 

In [ ]:
import boto3

from rasterio.plot import show
import rasterio as rio
from rasterio.session import AWSSession

aws_session = AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
aws_session

In [ ]:
def convert_llurl(ll_url: str) -> str:
    """
    Convert a landsat look url to an S3 url
    """
    return ll_url.replace('https://landsatlook.usgs.gov/data', 's3://usgs-landsat')

def open_dateset(ll_url: str):
    """
    Open a file with gdal
    """
    with rasterio.open(convert_llurl(ll_url)) as f:
        return f
#     return gdal.Open(path, gdal.ReadOnly)

In [ ]:
convert_llurl(filtered[0]['assets']['SR_B3.TIF']['href'])

In [ ]:
import xarray as xr

In [ ]:
with rio.Env(aws_session) as env:
    data = xr.open_rasterio(convert_llurl(filtered[0]['assets']['SR_B3.TIF']['href']))

In [ ]:
ds = open_dateset(filtered[0]['assets']['SR_B3.TIF']['href'])

# Time-series

# Change detection